In [1]:
pip install pyscipopt networkx scipy pandas matplotlib


Note: you may need to restart the kernel to use updated packages.


In [2]:
# STEP 1: Import libraries
import numpy as np
import pandas as pd
from pyscipopt import Model
from scipy.sparse import csr_matrix
from scipy.linalg import svd
from numpy.linalg import matrix_rank
import networkx as nx

# STEP 2: Load your MPS model
model = Model()
model.readProblem("gen-ip054.mps")  # Make sure the file is in the same directory

# STEP 3: Extract matrix A, RHS b, constraint senses
variables = model.getVars()
constraints = model.getConss()
var_names = [v.name for v in variables]
var_index = {name: idx for idx, name in enumerate(var_names)}
n_vars = len(variables)
n_cons = len(constraints)

A = np.zeros((n_cons, n_vars))
b = []
senses = []

for i, cons in enumerate(constraints):
    terms = model.getValsLinear(cons)
    for var_name in terms:
        j = var_index[var_name]
        A[i, j] = terms[var_name]
    lhs, rhs = model.getLhs(cons), model.getRhs(cons)
    if lhs == rhs:
        senses.append("=")
        b.append(rhs)
    elif np.isfinite(rhs):
        senses.append("<=")
        b.append(rhs)
    else:
        senses.append(">=")
        b.append(lhs)

# STEP 4: Convert A to sparse and get non-zero entries
A_sparse = csr_matrix(A)
A_dense = A
nonzeros = A_sparse.data

# STEP 5: Compute properties (structured from PDF)
summary = {}

# 1. Sparsity Metrics
summary["Shape"] = A.shape
summary["Sparsity (%)"] = 100 * (1 - A_sparse.nnz / (A.shape[0] * A.shape[1]))
summary["Row NNZ Variance"] = np.var(np.diff(A_sparse.indptr))
summary["Column NNZ Variance"] = np.var(np.diff(csr_matrix(A_sparse.T).indptr))

# 2. Coefficient Scale
summary["Min coefficient"] = np.min(nonzeros)
summary["Max coefficient"] = np.max(nonzeros)
summary["Mean coefficient"] = np.mean(nonzeros)
summary["Std coefficient"] = np.std(nonzeros)
summary["Integer-like (%)"] = 100 * np.sum(np.isclose(nonzeros % 1, 0, atol=1e-6)) / len(nonzeros)
summary["Largest abs coefficient"] = np.max(np.abs(nonzeros))
summary["Avg Row max/min ratio"] = np.mean([
    np.max(row[row != 0]) / np.min(np.abs(row[row != 0])) if np.any(row != 0) and np.min(np.abs(row[row != 0])) > 0 else 0
    for row in A_dense
])

# 3. Conditioning
U, s, Vt = svd(A_dense, full_matrices=False)
summary["Condition number"] = s[0] / s[-1] if s[-1] != 0 else np.inf
summary["Matrix rank"] = matrix_rank(A_dense)

# 4. Diagonal dominance (only if square)
if A.shape[0] == A.shape[1]:
    diag = np.abs(np.diag(A_dense))
    off_diag_sum = np.sum(np.abs(A_dense), axis=1) - diag
    summary["Diag Dominant Rows (%)"] = 100 * np.sum(diag > off_diag_sum) / A.shape[0]
else:
    summary["Diag Dominant Rows (%)"] = "N/A"

# 6. Norms
row_l2_norms = np.linalg.norm(A_dense, axis=1)
summary["Avg row L2 norm"] = np.mean(row_l2_norms)
summary["Max row L2 norm"] = np.max(row_l2_norms)
summary["Zero rows"] = np.sum(np.all(A_dense == 0, axis=1))
summary["Zero columns"] = np.sum(np.all(A_dense == 0, axis=0))

# 8. Graph metrics
G = nx.Graph()
G.add_nodes_from(range(A.shape[1]))
for i in range(A.shape[0]):
    cols = list(A_sparse.getrow(i).nonzero()[1])
    for j in range(len(cols)):
        for k in range(j + 1, len(cols)):
            G.add_edge(cols[j], cols[k])

degrees = [d for _, d in G.degree()]
summary["Avg degree"] = np.mean(degrees)
summary["Max degree"] = np.max(degrees)
summary["Clustering Coef (avg)"] = np.mean(list(nx.clustering(G).values()))
summary["Connected components"] = nx.number_connected_components(G)

# STEP 6: Display the summary
df_summary = pd.DataFrame(summary.items(), columns=["Property", "Value"])
df_summary


,Property,Value
0,Shape,"(27, 30)"
1,Sparsity (%),34.320988
2,Row NNZ Variance,49.02332
3,Column NNZ Variance,32.595556
4,Min coefficient,-13.584217
5,Max coefficient,1.706934
6,Mean coefficient,-2.749368
7,Std coefficient,3.554946
8,Integer-like (%),0.0
9,Largest abs coefficient,13.584217


##  Load MPS File and Extract Matrix A

In [3]:
# Load PySCIPOpt and extract matrix A
from pyscipopt import Model
import numpy as np

model = Model()
model.readProblem("gen-ip054.mps")

variables = model.getVars()
constraints = model.getConss()

var_names = [v.name for v in variables]
var_index = {name: idx for idx, name in enumerate(var_names)}

A = np.zeros((len(constraints), len(variables)))
b = []
senses = []

for i, cons in enumerate(constraints):
    terms = model.getValsLinear(cons)
    for var_name in terms:
        j = var_index[var_name]
        A[i, j] = terms[var_name]
    lhs, rhs = model.getLhs(cons), model.getRhs(cons)
    if lhs == rhs:
        senses.append("=")
        b.append(rhs)
    elif np.isfinite(rhs):
        senses.append("<=")
        b.append(rhs)
    else:
        senses.append(">=")
        b.append(lhs)

print(f"✅ Matrix A shape: {A.shape}")
print(f"Number of constraints: {len(constraints)}")
print(f"Number of variables: {len(variables)}")


✅ Matrix A shape: (27, 30)
Number of constraints: 27
Number of variables: 30


## Sparsity & Nonzero Patterns

In [4]:
from scipy.sparse import csr_matrix

A_sparse = csr_matrix(A)

total_entries = A.shape[0] * A.shape[1]
nnz = A_sparse.nnz
sparsity_percent = 100 * (1 - nnz / total_entries)

print("🔍 Sparsity Analysis:")
print(f" - Total entries: {total_entries}")
print(f" - Non-zero entries: {nnz}")
print(f" - Sparsity: {sparsity_percent:.2f}%")

# Compute bandwidth: max(|i - j|) for all A[i, j] ≠ 0
row_indices, col_indices = A_sparse.nonzero()
bandwidth = np.max(np.abs(row_indices - col_indices))

print("📏 Bandwidth Analysis:")
print(f" - Matrix bandwidth: {bandwidth}")



🔍 Sparsity Analysis:
 - Total entries: 810
 - Non-zero entries: 532
 - Sparsity: 34.32%
📏 Bandwidth Analysis:
 - Matrix bandwidth: 28


## Coefficient Statistics

In [5]:
nonzeros = A_sparse.data

print("🔢 Coefficient Stats:")
print(f" - Min coefficient: {np.min(nonzeros)}")
print(f" - Max coefficient: {np.max(nonzeros)}")
print(f" - Mean coefficient: {np.mean(nonzeros):.4f}")
print(f" - Std deviation: {np.std(nonzeros):.4f}")
print(f" - Largest abs coefficient: {np.max(np.abs(nonzeros))}")


🔢 Coefficient Stats:
 - Min coefficient: -13.5842166
 - Max coefficient: 1.706933862
 - Mean coefficient: -2.7494
 - Std deviation: 3.5549
 - Largest abs coefficient: 13.5842166


##  Integer-Like Coefficients

In [6]:
int_like_count = np.sum(np.isclose(nonzeros % 1, 0, atol=1e-6))
int_ratio = 100 * int_like_count / len(nonzeros)

print("🔢 Integer Check:")
print(f" - Integer-like values: {int_like_count}")
print(f" - Integer-like ratio: {int_ratio:.2f}%")


🔢 Integer Check:
 - Integer-like values: 0
 - Integer-like ratio: 0.00%


## Row Scaling – Max/Min Ratio

In [7]:
row_ratios = []
for row in A:
    nz = row[row != 0]
    if len(nz) > 1 and np.min(np.abs(nz)) > 0:
        row_ratios.append(np.max(np.abs(nz)) / np.min(np.abs(nz)))

print("📏 Row Scaling Ratios:")
print(f" - Avg max/min ratio: {np.mean(row_ratios):.2f}")
print(f" - Max max/min ratio: {np.max(row_ratios):.2f}")


📏 Row Scaling Ratios:
 - Avg max/min ratio: 26.89
 - Max max/min ratio: 179.02


##  Matrix Rank and Condition Number


In [8]:
from scipy.linalg import svd
from numpy.linalg import matrix_rank

U, s, Vt = svd(A, full_matrices=False)
cond_number = s[0] / s[-1] if s[-1] != 0 else np.inf
rank_val = matrix_rank(A)

print("🔬 Linear Algebra:")
print(f" - Matrix rank: {rank_val}")
print(f" - Condition number: {cond_number:.2e}")


🔬 Linear Algebra:
 - Matrix rank: 27
 - Condition number: 1.74e+02


In [9]:
# Explanation:
#Rank shows if constraints are redundant.
#Condition number tells how numerically stable the matrix is.



## Norms and Structure

In [10]:
l2_norms = np.linalg.norm(A, axis=1)
zero_rows = np.sum(np.all(A == 0, axis=1))
zero_cols = np.sum(np.all(A == 0, axis=0))

print("📐 Norms & Zeros:")
print(f" - Avg row L2 norm: {np.mean(l2_norms):.2f}")
print(f" - Max row L2 norm: {np.max(l2_norms):.2f}")
print(f" - Zero rows: {zero_rows}")
print(f" - Zero columns: {zero_cols}")


📐 Norms & Zeros:
 - Avg row L2 norm: 12.90
 - Max row L2 norm: 60.17
 - Zero rows: 0
 - Zero columns: 0


## Build Variable Interaction Graph

In [11]:
import networkx as nx

# Create an undirected graph
G = nx.Graph()
G.add_nodes_from(range(A.shape[1]))  # one node per variable

# Add edges for co-occurring variables in each constraint
for i in range(A.shape[0]):
    cols = list(A_sparse.getrow(i).nonzero()[1])
    for j in range(len(cols)):
        for k in range(j + 1, len(cols)):
            G.add_edge(cols[j], cols[k])

print("✅ Graph built from constraint matrix")
print(f" - Total variables (nodes): {G.number_of_nodes()}")
print(f" - Total interactions (edges): {G.number_of_edges()}")

#This graph models variable-variable interactions, which lets us apply graph theory:
#Higher connectivity may mean denser subproblems
#Sparsity can help decomposition

✅ Graph built from constraint matrix
 - Total variables (nodes): 30
 - Total interactions (edges): 435


## Analyze Graph Properties

In [12]:
# Degree stats
degrees = [d for _, d in G.degree()]
avg_degree = np.mean(degrees)
max_degree = np.max(degrees)

# Clustering and connectivity
clustering_coeff = nx.average_clustering(G)
connected_components = nx.number_connected_components(G)

print("📊 Graph Metrics:")
print(f" - Average node degree: {avg_degree:.2f}")
print(f" - Maximum node degree: {max_degree}")
print(f" - Average clustering coefficient: {clustering_coeff:.4f}")
print(f" - Number of connected components: {connected_components}")

#Degree	                 : How many constraints involve each variable
#Clustering	Local density: are groups of variables tightly coupled?
#Connected components	 : Disconnected blocks = potential decomposable submodels


📊 Graph Metrics:
 - Average node degree: 29.00
 - Maximum node degree: 29
 - Average clustering coefficient: 1.0000
 - Number of connected components: 1


## Visualize (small graph only)


In [13]:
import matplotlib.pyplot as plt

# Draw subgraph for first 50 variables
nodes_subset = list(G.nodes)[:50]
subG = G.subgraph(nodes_subset)

#plt.figure(figsize=(8, 6))
#nx.draw(subG, with_labels=True, node_size=300, node_color='skyblue', font_size=8)
#plt.title("🔗 Variable Interaction Subgraph (first 50 variables)")
#plt.show()


## Short Summarey

In [14]:
import pandas as pd


In [15]:
# Suppose df_summary holds your summary
df_summary = pd.DataFrame([
    ["Shape", A.shape],
    ["Sparsity (%)", sparsity_percent],
    ["Non-zero entries", A_sparse.nnz],
    ["Min coefficient", np.min(nonzeros)],
    ["Max coefficient", np.max(nonzeros)],
    ["Integer-like (%)", int_ratio],
    ["Condition number", cond_number],
    ["Matrix rank", rank_val],
    ["Avg row L2 norm", np.mean(l2_norms)],
    ["Zero rows", zero_rows],
    ["Zero columns", zero_cols],
    ["Avg degree", avg_degree],
    ["Clustering Coefficient", clustering_coeff],
    ["Connected components", connected_components]
], columns=["Property", "Value"])


## 1. Sparsity-Related Metrics 

In [16]:
# 📦 Required imports
import numpy as np
from scipy.sparse import csr_matrix

# 🧮 Convert matrix A to sparse format (assuming you already loaded A)
A_sparse = csr_matrix(A)

# 🔹 Step 1: Sparsity (% of zero entries)
total_entries = A_sparse.shape[0] * A_sparse.shape[1]
non_zero_entries = A_sparse.nnz
sparsity = 100 * (1 - non_zero_entries / total_entries)

print("🔷 1. Sparsity-Related Metrics\n")
print(f"📏 Matrix shape: {A_sparse.shape[0]} rows x {A_sparse.shape[1]} columns")
print(f"🔹 Total entries: {total_entries}")
print(f"🔹 Non-zero entries: {non_zero_entries}")
print(f"🔹 Sparsity: {sparsity:.2f}%")
print("🧠 Explanation: Sparsity is the % of zero entries in the matrix A. High sparsity is common in optimization problems.\n")

# 🔹 Step 2: Row/Column Nonzero Counts
row_nnz = np.diff(A_sparse.indptr)
col_nnz = np.diff(csr_matrix(A_sparse.T).indptr)

print(f"🔸 Average non-zero entries per row: {np.mean(row_nnz):.2f}")
print(f"🔸 Average non-zero entries per column: {np.mean(col_nnz):.2f}")
print("🧠 Explanation: These counts show how dense each row/column is, which affects solver effort.\n")

# 🔹 Step 3: Row/Column Density Variance
row_var = np.var(row_nnz)
col_var = np.var(col_nnz)

print(f"🔸 Variance of non-zero entries across rows: {row_var:.2f}")
print(f"🔸 Variance of non-zero entries across columns: {col_var:.2f}")
print("🧠 Explanation: High variance means some rows/columns are much denser than others — this can cause imbalance in the solver's workload.\n")

# 🔹 Step 4: Bandwidth
row_indices, col_indices = A_sparse.nonzero()
bandwidth = np.max(np.abs(row_indices - col_indices))

print(f"🔸 Matrix Bandwidth: {bandwidth}")
print("🧠 Explanation: Bandwidth measures how far non-zero entries lie from the diagonal. Lower bandwidth = better for solver reordering.\n")

print("✅ All Sparsity-Related Metrics Computed.")


🔷 1. Sparsity-Related Metrics

📏 Matrix shape: 27 rows x 30 columns
🔹 Total entries: 810
🔹 Non-zero entries: 532
🔹 Sparsity: 34.32%
🧠 Explanation: Sparsity is the % of zero entries in the matrix A. High sparsity is common in optimization problems.

🔸 Average non-zero entries per row: 19.70
🔸 Average non-zero entries per column: 17.73
🧠 Explanation: These counts show how dense each row/column is, which affects solver effort.

🔸 Variance of non-zero entries across rows: 49.02
🔸 Variance of non-zero entries across columns: 32.60
🧠 Explanation: High variance means some rows/columns are much denser than others — this can cause imbalance in the solver's workload.

🔸 Matrix Bandwidth: 28
🧠 Explanation: Bandwidth measures how far non-zero entries lie from the diagonal. Lower bandwidth = better for solver reordering.

✅ All Sparsity-Related Metrics Computed.


##  2. Coefficient Size & Scale 

In [17]:
# 📦 Required imports
import numpy as np

# 🔹 Ensure you’re working with sparse matrix
nonzeros = A_sparse.data

print("🔷 2. Coefficient Size & Scale Metrics\n")

# 🔸 1. Basic Stats
print(f"📊 Total non-zero coefficients: {len(nonzeros)}")
print(f"🔹 Min value: {np.min(nonzeros)}")
print(f"🔹 Max value: {np.max(nonzeros)}")
print(f"🔹 Mean value: {np.mean(nonzeros):.4f}")
print(f"🔹 Standard deviation: {np.std(nonzeros):.4f}")
print("🧠 Explanation: These describe the spread of coefficients in A. Large values or big variance may slow solver performance.\n")

# 🔸 2. Row-wise max/min ratio (scaling)
row_ratios = []
for row in A:
    nz = row[row != 0]
    if len(nz) > 1 and np.min(np.abs(nz)) > 0:
        row_ratios.append(np.max(np.abs(nz)) / np.min(np.abs(nz)))

print(f"🔹 Avg row-wise max/min ratio: {np.mean(row_ratios):.2f}")
print(f"🔹 Max row-wise max/min ratio: {np.max(row_ratios):.2f}")
print("🧠 Explanation: This highlights scaling issues — rows with large ratio can cause numerical instability.\n")

# 🔸 3. Largest absolute coefficient
print(f"🔹 Largest absolute coefficient: {np.max(np.abs(nonzeros))}")
print("🧠 Explanation: Large absolute values affect solver tolerances. Try normalizing if this is very high.\n")

# 🔸 4. Fraction of integer vs fractional coefficients
integer_like = np.sum(np.isclose(nonzeros % 1, 0, atol=1e-6))
fractional = len(nonzeros) - integer_like
integer_ratio = 100 * integer_like / len(nonzeros)

print(f"🔹 Integer-like coefficients: {integer_like} ({integer_ratio:.2f}%)")
print(f"🔹 Fractional coefficients: {fractional} ({100 - integer_ratio:.2f}%)")
print("🧠 Explanation: Pure IP models often use integer coefficients. Fractional values suggest a scaled or mixed-integer model.\n")

print("✅ Coefficient Size & Scale Metrics Computed.")


🔷 2. Coefficient Size & Scale Metrics

📊 Total non-zero coefficients: 532
🔹 Min value: -13.5842166
🔹 Max value: 1.706933862
🔹 Mean value: -2.7494
🔹 Standard deviation: 3.5549
🧠 Explanation: These describe the spread of coefficients in A. Large values or big variance may slow solver performance.

🔹 Avg row-wise max/min ratio: 26.89
🔹 Max row-wise max/min ratio: 179.02
🧠 Explanation: This highlights scaling issues — rows with large ratio can cause numerical instability.

🔹 Largest absolute coefficient: 13.5842166
🧠 Explanation: Large absolute values affect solver tolerances. Try normalizing if this is very high.

🔹 Integer-like coefficients: 0 (0.00%)
🔹 Fractional coefficients: 532 (100.00%)
🧠 Explanation: Pure IP models often use integer coefficients. Fractional values suggest a scaled or mixed-integer model.

✅ Coefficient Size & Scale Metrics Computed.


## 3. Condition-Related 

In [18]:
# 📦 Required imports
from scipy.linalg import svd
from numpy.linalg import matrix_rank

print("🔷 3. Condition-Related Metrics\n")

# 🔸 1. Condition Number (via SVD)
U, s, Vt = svd(A, full_matrices=False)
cond_number = s[0] / s[-1] if s[-1] > 0 else np.inf

print(f"🔹 Condition Number: {cond_number:.2e}")
print("🧠 Explanation: The condition number is the ratio of the largest to smallest singular values.")
print("   A large number (e.g. >1e5) suggests ill-conditioning → may cause instability for solvers.\n")

# 🔸 2. Matrix Rank
rank_val = matrix_rank(A)
print(f"🔹 Matrix Rank: {rank_val}")
print("🧠 Explanation: Rank indicates linear independence. If rank < number of rows, the system is redundant or overconstrained.\n")

# 🔸 3. Diagonal Dominance (only applies if square matrix)
if A.shape[0] == A.shape[1]:
    diag = np.abs(np.diag(A))
    off_diag_sum = np.sum(np.abs(A), axis=1) - diag
    diag_dominant_rows = np.sum(diag > off_diag_sum)
    dominance_percent = 100 * diag_dominant_rows / A.shape[0]

    print(f"🔹 Diagonally Dominant Rows: {diag_dominant_rows} ({dominance_percent:.2f}%)")
    print("🧠 Explanation: A row is diagonally dominant if its diagonal entry is larger than the sum of all other entries.")
    print("   This improves numerical stability. Often checked in linear system solvers.\n")
else:
    print("⚠️ Diagonal dominance not computed (matrix is not square).")
    print("🧠 Explanation: Diagonal dominance only applies to square matrices (same # constraints as variables).\n")

print("✅ Condition-Related Metrics Computed.")


🔷 3. Condition-Related Metrics

🔹 Condition Number: 1.74e+02
🧠 Explanation: The condition number is the ratio of the largest to smallest singular values.
   A large number (e.g. >1e5) suggests ill-conditioning → may cause instability for solvers.

🔹 Matrix Rank: 27
🧠 Explanation: Rank indicates linear independence. If rank < number of rows, the system is redundant or overconstrained.

⚠️ Diagonal dominance not computed (matrix is not square).
🧠 Explanation: Diagonal dominance only applies to square matrices (same # constraints as variables).

✅ Condition-Related Metrics Computed.


## 4. Structural Graph-Theoretic Metrics (via networkx)

In [19]:
# 📦 Required imports
import networkx as nx
import matplotlib.pyplot as plt

print("🔷 4. Structural Graph-Theoretic Metrics\n")

# 🔸 Step 1: Build Variable Interaction Graph
# Each node = variable (column); edges = co-occurrence in a constraint (row)
G = nx.Graph()
G.add_nodes_from(range(A.shape[1]))

for i in range(A.shape[0]):
    cols = list(A_sparse.getrow(i).nonzero()[1])
    for j in range(len(cols)):
        for k in range(j + 1, len(cols)):
            G.add_edge(cols[j], cols[k])

print(f"📦 Graph built with {G.number_of_nodes()} variables (nodes) and {G.number_of_edges()} edges (co-occurrence links)\n")

# 🔸 Step 2: Degree Distribution
degrees = [deg for _, deg in G.degree()]
avg_degree = np.mean(degrees)
max_degree = np.max(degrees)

print(f"🔹 Average node degree: {avg_degree:.2f}")
print(f"🔹 Maximum node degree: {max_degree}")
print("🧠 Explanation: High-degree nodes are involved in many constraints — they are bottlenecks or hubs.\n")

# 🔸 Step 3: Clustering Coefficient
clustering = nx.clustering(G)
clustering_avg = np.mean(list(clustering.values()))

print(f"🔹 Average clustering coefficient: {clustering_avg:.4f}")
print("🧠 Explanation: Measures how connected a node's neighbors are. High values = dense variable neighborhoods.\n")

# 🔸 Step 4: Connected Components
components = list(nx.connected_components(G))
num_components = len(components)
component_sizes = [len(c) for c in components]
largest_component = max(component_sizes)

print(f"🔹 Number of connected components: {num_components}")
print(f"🔹 Largest component size: {largest_component}")
print("🧠 Explanation: Disconnected subgraphs can be optimized independently — useful for decomposition.\n")

# 🔸 Step 5 (Optional): Treewidth (heuristic via min fill-in)
try:
    from networkx.algorithms.approximation.treewidth import treewidth_min_fill_in
    tw, _ = treewidth_min_fill_in(G)
    print(f"🔹 Treewidth (approximate): {tw}")
    print("🧠 Explanation: Low treewidth → better solver performance due to decomposability.\n")
except:
    print("⚠️ Treewidth estimation not available — requires `networkx >= 2.6`.")
    print("🧠 Explanation: Treewidth is NP-hard to compute exactly, so approximation is used.\n")

print("✅ Structural Graph-Theoretic Metrics Computed.")


🔷 4. Structural Graph-Theoretic Metrics

📦 Graph built with 30 variables (nodes) and 435 edges (co-occurrence links)

🔹 Average node degree: 29.00
🔹 Maximum node degree: 29
🧠 Explanation: High-degree nodes are involved in many constraints — they are bottlenecks or hubs.

🔹 Average clustering coefficient: 1.0000
🧠 Explanation: Measures how connected a node's neighbors are. High values = dense variable neighborhoods.

🔹 Number of connected components: 1
🔹 Largest component size: 30
🧠 Explanation: Disconnected subgraphs can be optimized independently — useful for decomposition.

🔹 Treewidth (approximate): 29
🧠 Explanation: Low treewidth → better solver performance due to decomposability.

✅ Structural Graph-Theoretic Metrics Computed.


## 5. Constraint Classification 

In [20]:
print("🔷 5. Constraint Classification\n")

# 🔸 1. Equalities & Inequalities
equalities = sum(1 for i in range(len(b)) if senses[i] == "=")
inequalities = len(b) - equalities

print(f"🔹 Equalities (lhs == rhs): {equalities}")
print(f"🔹 Inequalities (lhs ≠ rhs): {inequalities}")
print("🧠 Explanation: Equalities are exact constraints. Inequalities allow room on one side and often involve slacks.\n")

# 🔸 2. Bounds-only variables
used_vars = set(A_sparse.nonzero()[1])
all_vars = set(range(A_sparse.shape[1]))
bound_only_vars = all_vars - used_vars

print(f"🔹 Variables appearing only in bounds (not in constraints): {len(bound_only_vars)}")
print("🧠 Explanation: These variables don't participate in any constraint — they are just bounded.\n")

# 🔸 3. Slack variables: exactly one row, ±1 coefficient
slack_vars = 0
for j in range(A_sparse.shape[1]):
    col_vals = A_sparse.getcol(j).toarray().flatten()
    nonzeros = col_vals[col_vals != 0]
    if len(nonzeros) == 1 and np.isclose(abs(nonzeros[0]), 1.0, atol=1e-6):
        slack_vars += 1

print(f"🔹 Slack-like variables (only one ±1 entry): {slack_vars}")
print("🧠 Explanation: Slack variables are added to turn inequalities into equalities and appear exactly once with ±1.\n")

# 🔸 4. Redundant constraints (check rank)
from numpy.linalg import matrix_rank

num_constraints = A.shape[0]
rank_val = matrix_rank(A)
redundant = num_constraints - rank_val

print(f"🔹 Redundant constraints (based on matrix rank): {redundant}")
print(f"   - Matrix rank: {rank_val}")
print(f"   - Total constraints: {num_constraints}")
print("🧠 Explanation: Redundant constraints do not add new information and may slow down the solver.\n")

print("✅ Constraint classification completed.")


🔷 5. Constraint Classification

🔹 Equalities (lhs == rhs): 0
🔹 Inequalities (lhs ≠ rhs): 27
🧠 Explanation: Equalities are exact constraints. Inequalities allow room on one side and often involve slacks.

🔹 Variables appearing only in bounds (not in constraints): 0
🧠 Explanation: These variables don't participate in any constraint — they are just bounded.

🔹 Slack-like variables (only one ±1 entry): 0
🧠 Explanation: Slack variables are added to turn inequalities into equalities and appear exactly once with ±1.

🔹 Redundant constraints (based on matrix rank): 0
   - Matrix rank: 27
   - Total constraints: 27
🧠 Explanation: Redundant constraints do not add new information and may slow down the solver.

✅ Constraint classification completed.


## 6. Normalized Coefficient Statistics

In [21]:
print("🔷 6. Normalized Coefficient Statistics\n")

# 🔸 1. Row-wise Norms
row_l2_norms = np.linalg.norm(A, axis=1)
row_l1_norms = np.sum(np.abs(A), axis=1)

print(f"🔹 Avg row L2 norm: {np.mean(row_l2_norms):.2f}")
print(f"🔹 Max row L2 norm: {np.max(row_l2_norms):.2f}")
print(f"🔹 Avg row L1 norm: {np.mean(row_l1_norms):.2f}")
print(f"🔹 Max row L1 norm: {np.max(row_l1_norms):.2f}")
print("🧠 Explanation: L1/L2 norms help detect scaling problems across constraints.\n")

# 🔸 2. Coefficient range after normalization (per row)
normalized_ranges = []
for row in A:
    abs_row = np.abs(row[row != 0])
    if len(abs_row) > 1:
        rng = np.max(abs_row) / np.min(abs_row)
        normalized_ranges.append(rng)

print(f"🔹 Avg normalized coefficient range (row): {np.mean(normalized_ranges):.2f}")
print(f"🔹 Max normalized coefficient range (row): {np.max(normalized_ranges):.2f}")
print("🧠 Explanation: Big ranges between smallest/largest coefficient show uneven constraint scaling.\n")

# 🔸 3. Heavy Constraints – Sum of Absolute Values per Row
heavy_threshold = np.percentile(row_l1_norms, 95)
heavy_constraints = np.sum(row_l1_norms > heavy_threshold)

print(f"🔹 Constraints with high total weight (> 95th percentile): {heavy_constraints}")
print("🧠 Explanation: Heavy constraints may dominate solver behavior — watch for imbalance.\n")

# 🔸 4. Zero row/column detection
zero_rows = np.sum(np.all(A == 0, axis=1))
zero_cols = np.sum(np.all(A == 0, axis=0))

print(f"🔹 Zero rows (unused constraints): {zero_rows}")
print(f"🔹 Zero columns (unused variables): {zero_cols}")
print("🧠 Explanation: Zero rows or columns can usually be removed — they don't affect the model.\n")

print("✅ Normalized Coefficient Statistics Computed.")


🔷 6. Normalized Coefficient Statistics

🔹 Avg row L2 norm: 12.90
🔹 Max row L2 norm: 60.17
🔹 Avg row L1 norm: 56.57
🔹 Max row L1 norm: 306.29
🧠 Explanation: L1/L2 norms help detect scaling problems across constraints.

🔹 Avg normalized coefficient range (row): 26.89
🔹 Max normalized coefficient range (row): 179.02
🧠 Explanation: Big ranges between smallest/largest coefficient show uneven constraint scaling.

🔹 Constraints with high total weight (> 95th percentile): 2
🧠 Explanation: Heavy constraints may dominate solver behavior — watch for imbalance.

🔹 Zero rows (unused constraints): 0
🔹 Zero columns (unused variables): 0
🧠 Explanation: Zero rows or columns can usually be removed — they don't affect the model.

✅ Normalized Coefficient Statistics Computed.


## 7. Redundancy and Linearity

In [22]:
print("🔷 6. Normalized Coefficient Statistics\n")

# 🔸 1. Row-wise Norms
row_l2_norms = np.linalg.norm(A, axis=1)
row_l1_norms = np.sum(np.abs(A), axis=1)

print(f"🔹 Avg row L2 norm: {np.mean(row_l2_norms):.2f}")
print(f"🔹 Max row L2 norm: {np.max(row_l2_norms):.2f}")
print(f"🔹 Avg row L1 norm: {np.mean(row_l1_norms):.2f}")
print(f"🔹 Max row L1 norm: {np.max(row_l1_norms):.2f}")
print("🧠 Explanation: L1/L2 norms help detect scaling problems across constraints.\n")

# 🔸 2. Coefficient range after normalization (per row)
normalized_ranges = []
for row in A:
    abs_row = np.abs(row[row != 0])
    if len(abs_row) > 1:
        rng = np.max(abs_row) / np.min(abs_row)
        normalized_ranges.append(rng)

print(f"🔹 Avg normalized coefficient range (row): {np.mean(normalized_ranges):.2f}")
print(f"🔹 Max normalized coefficient range (row): {np.max(normalized_ranges):.2f}")
print("🧠 Explanation: Big ranges between smallest/largest coefficient show uneven constraint scaling.\n")

# 🔸 3. Heavy Constraints – Sum of Absolute Values per Row
heavy_threshold = np.percentile(row_l1_norms, 95)
heavy_constraints = np.sum(row_l1_norms > heavy_threshold)

print(f"🔹 Constraints with high total weight (> 95th percentile): {heavy_constraints}")
print("🧠 Explanation: Heavy constraints may dominate solver behavior — watch for imbalance.\n")

# 🔸 4. Zero row/column detection
zero_rows = np.sum(np.all(A == 0, axis=1))
zero_cols = np.sum(np.all(A == 0, axis=0))

print(f"🔹 Zero rows (unused constraints): {zero_rows}")
print(f"🔹 Zero columns (unused variables): {zero_cols}")
print("🧠 Explanation: Zero rows or columns can usually be removed — they don't affect the model.\n")

print("✅ Normalized Coefficient Statistics Computed.")


🔷 6. Normalized Coefficient Statistics

🔹 Avg row L2 norm: 12.90
🔹 Max row L2 norm: 60.17
🔹 Avg row L1 norm: 56.57
🔹 Max row L1 norm: 306.29
🧠 Explanation: L1/L2 norms help detect scaling problems across constraints.

🔹 Avg normalized coefficient range (row): 26.89
🔹 Max normalized coefficient range (row): 179.02
🧠 Explanation: Big ranges between smallest/largest coefficient show uneven constraint scaling.

🔹 Constraints with high total weight (> 95th percentile): 2
🧠 Explanation: Heavy constraints may dominate solver behavior — watch for imbalance.

🔹 Zero rows (unused constraints): 0
🔹 Zero columns (unused variables): 0
🧠 Explanation: Zero rows or columns can usually be removed — they don't affect the model.

✅ Normalized Coefficient Statistics Computed.


## 8. Variable Participation Statistics 

In [23]:
print("🔷 8. Variable Participation Statistics\n")

# 1. Constraints per variable (column nonzero count)
col_nnz = np.diff(csr_matrix(A_sparse.T).indptr)
avg_participation = np.mean(col_nnz)
max_participation = np.max(col_nnz)

print(f"🔹 Avg # constraints per variable: {avg_participation:.2f}")
print(f"🔹 Max # constraints per variable: {max_participation}")
print("🧠 Explanation: Highly connected variables may become bottlenecks in decomposition.\n")

# 2. Binary/Integer variable density (using PySCIPOpt variable type)
integer_vars = [v for v in variables if v.vtype() in ("BINARY", "INTEGER")]
integer_ratio = 100 * len(integer_vars) / len(variables)

print(f"🔹 Integer or Binary variables: {len(integer_vars)} ({integer_ratio:.2f}%)")
print("🧠 Explanation: Mixed-integer models contain a mix of discrete and continuous variables.\n")

# 4. Slack variable detection
slack_count = 0
for j in range(A_sparse.shape[1]):
    col_vals = A_sparse.getcol(j).toarray().flatten()
    nonzeros = col_vals[col_vals != 0]
    if len(nonzeros) == 1 and np.isclose(abs(nonzeros[0]), 1.0, atol=1e-6):
        slack_count += 1

print(f"🔹 Slack-like variables (1 row, ±1): {slack_count}")
print("🧠 Explanation: Slack variables are auxiliary variables added to reformulate constraints.\n")

print("✅ Variable Participation Statistics Computed.")


🔷 8. Variable Participation Statistics

🔹 Avg # constraints per variable: 17.73
🔹 Max # constraints per variable: 24
🧠 Explanation: Highly connected variables may become bottlenecks in decomposition.

🔹 Integer or Binary variables: 30 (100.00%)
🧠 Explanation: Mixed-integer models contain a mix of discrete and continuous variables.

🔹 Slack-like variables (1 row, ±1): 0
🧠 Explanation: Slack variables are auxiliary variables added to reformulate constraints.

✅ Variable Participation Statistics Computed.


## 9. Constraint Types by Pattern 


In [24]:
print("🔷 9. Constraint Types by Pattern\n")

from collections import Counter

# Initialize counters for each constraint type
pattern_counts = Counter({
    "Knapsack": 0,
    "Packing/Covering": 0,
    "Flow Conservation": 0,
    "Assignment": 0,
    "Set Partitioning": 0
})

# Loop over all rows in A and analyze patterns
for i in range(A.shape[0]):
    row = A[i]
    nz = row[row != 0]
    abs_vals = np.abs(nz)
    signs = np.sign(nz)

    # For set-partitioning: binary vars only, rhs = 1
    if all(v in [0, 1] for v in abs_vals) and np.isclose(b[i], 1) and senses[i] == "=":
        pattern_counts["Set Partitioning"] += 1

    # For assignment: only one nonzero and it's 1
    if np.count_nonzero(row) == 1 and np.isclose(nz[0], 1):
        pattern_counts["Assignment"] += 1

    # Flow conservation: exactly one +1 and one -1
    if list(signs).count(1) == 1 and list(signs).count(-1) == 1 and len(signs) == 2:
        pattern_counts["Flow Conservation"] += 1

    # Knapsack: all positive and ≤ 1
    if np.all(nz > 0) and np.all(nz <= 1):
        pattern_counts["Knapsack"] += 1

    # Packing/Covering: 0/1 coefficients and sense ≤ or ≥
    if all(v in [0, 1] for v in abs_vals) and senses[i] in ["<=", ">="]:
        pattern_counts["Packing/Covering"] += 1

# Print summary
for pattern, count in pattern_counts.items():
    print(f"🔹 {pattern}: {count} constraints")

print("\n🧠 Explanation:")
print("- These patterns often occur in structured models (e.g. flows, assignments).")
print("- Recognizing them allows solvers to apply special preprocessing or decomposition.")
print("✅ Constraint pattern detection complete.")


🔷 9. Constraint Types by Pattern

🔹 Knapsack: 0 constraints
🔹 Packing/Covering: 0 constraints
🔹 Flow Conservation: 0 constraints
🔹 Assignment: 0 constraints
🔹 Set Partitioning: 0 constraints

🧠 Explanation:
- These patterns often occur in structured models (e.g. flows, assignments).
- Recognizing them allows solvers to apply special preprocessing or decomposition.
✅ Constraint pattern detection complete.


In [25]:
def compute_matrix_statistics(self, A_sparse):
    import pandas as pd

    rows, cols = A_sparse.shape
    total_entries = rows * cols
    nonzeros = A_sparse.nnz
    sparsity = 1 - nonzeros / total_entries

    row_nnz = np.asarray((A_sparse != 0).sum(axis=1)).flatten()
    col_nnz = np.asarray((A_sparse != 0).sum(axis=0)).flatten()

    row_var = np.var(row_nnz)
    col_var = np.var(col_nnz)

    row_idx, col_idx = A_sparse.nonzero()
    bandwidth = np.max(np.abs(row_idx - col_idx)) if len(row_idx) else 0

    # Integer-like values check
    values = A_sparse.data
    is_integer_like = np.isclose(values % 1, 0, atol=1e-6)
    integer_ratio = np.sum(is_integer_like) / len(values)

    stats = {
        "Sparsity (%)": [100 * sparsity],
        "Avg Nonzeros per Row": [np.mean(row_nnz)],
        "Avg Nonzeros per Column": [np.mean(col_nnz)],
        "Row Density Variance": [row_var],
        "Column Density Variance": [col_var],
        "Matrix Bandwidth": [bandwidth],
        "Integer-Like Coefficient %": [100 * integer_ratio],
        "Total Nonzeros": [nonzeros]
    }

    return pd.DataFrame(stats)


In [ ]:
pip install PySide6


In [ ]:
from PySide6.QtWidgets import QTableWidget, QTableWidgetItem


In [ ]:
from PySide6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QTextEdit, QFileDialog,
    QPushButton, QTableWidget, QTableWidgetItem
)


In [ ]:
class MPSLoaderApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("MPS Loader and Sparse Matrix Viewer")
        self.resize(800, 600)

        self.matrix_stats = None  # ✅ Add it here

